In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Repair archive
# sudo zip -FF T162.zip --out T162Repaired.zip

In [3]:
from typing import List
from datasetsforecast.long_horizon import LongHorizon
import pandas as pd
import numpy as np
import torch
from typing import Optional

import os, pathlib
from glob import glob

from torch.utils.data import Dataset, DataLoader, Subset
from tqdm.auto import tqdm

from dataset import LongHorizonUnivariateDataModule, LongHorizonUnivariateDataset
from dataset import ElectricityUnivariateDataModule, ElectricityUnivariateDataset

import pytorch_lightning as pl
from omegaconf import OmegaConf
from utils.model_factory import instantiate

from onedrivedownloader import download

from statsforecast.models import AutoETS, ETS, Theta, AutoCES

from metrics import SMAPE, MAPE, CRPS

RESULTS_DIR = './results'
# MANDATORY_TESTING_QUANTS = [0.5, 0.001, 0.01, 0.05, 0.1, 0.25, 0.75, 0.9, 0.95, 0.99, 0.999]
MANDATORY_TESTING_QUANTS = np.concatenate([[0.5], [0.001], np.arange(0.01,0.5,0.01), np.arange(0.51,1,0.01), [0.999]]) 

/opt/conda/lib/python3.10/site-packages/lightning_fabric/__init__.py:29: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)
/opt/conda/lib/python3.10/site-packages/statsforecast/utils.py:231: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  "ds": pd.date_range(start="1949-01-01", periods=len(AirPassengers), freq="M"),


In [4]:
def marfe(df, actuals_col="actuals", predictions_col="predictions", quantiles_col="quants", bin_size=0.001):
    df_cpy = df[[actuals_col, predictions_col, quantiles_col]].reset_index(drop=True).copy()
    
    df_cpy["quants_bins"] = np.round(df_cpy[quantiles_col], int(np.log10(1/bin_size))).values
    df_cpy["ReFr"] = df_cpy[actuals_col] < df_cpy[predictions_col]
    refr = df_cpy.groupby('quants_bins')["ReFr"].mean()
    # refr.drop([0.0, 1.0], inplace=True)
    return np.mean(np.abs(refr.values - refr.index))

In [5]:
# epoch = 4
# root_path = os.path.join(RESULTS_DIR, 'MHLV/slawek')

epoch = 1
root_path = os.path.join(RESULTS_DIR, 'MHLV/NBEATSAQFILM-maxnorm=False-loss=MQNLoss')

crps_rnd = CRPS()
crps_fixed = CRPS()
mape = MAPE()

tot_len = 0
num_nulls = {}

dfs_all = []
marfe_ts = []
dfs_all_fixed = []
df_quants_ts = []
for ts in tqdm(range(35)):
    paths = glob(os.path.join(root_path, f'e{epoch}w*_{ts}.pickle'))
    df = pd.concat([pd.read_pickle(p) for p in paths], axis=1)
    # Remove rows corresponding to the nan values in actuals, implying invalid ground truth observations
    num_nulls[ts] = num_nulls.get(ts, 0) + df.actuals.isnull().sum()
    df = df[~df.actuals.isnull()]
    
    df['predictions'] = df[df.columns[df.columns.str.startswith('forec')]].mean(axis=1).values
    
    mandatory_quants = df.quants.isin(np.array(MANDATORY_TESTING_QUANTS))
    df_rnd = df[~mandatory_quants]
    df_fixed = df[mandatory_quants]
    
    crps_rnd.update(preds=torch.Tensor(df_rnd['predictions'].values)[None], 
                    target=torch.Tensor(df_rnd['actuals'].values)[None], 
                    q=torch.Tensor(df_rnd['quants'].values)[None])
    
    crps_fixed.update(preds=torch.Tensor(df_fixed['predictions'].values)[None], 
                      target=torch.Tensor(df_fixed['actuals'].values)[None], 
                      q=torch.Tensor(df_fixed['quants'].values)[None])
    
    
    median_quants = df_fixed.quants.isin(np.array([0.5]))
    mape.update(target=torch.Tensor(df_fixed['actuals'][median_quants].values),
                preds=torch.Tensor(df_fixed['predictions'][median_quants].values))
    
    tot_len += len(df)
    dfs_all.append(df_rnd)
    dfs_all_fixed.append(df_fixed)

    marfe_ts.append(marfe(df_fixed, bin_size=0.01))

    df_quants = df_fixed[['actuals','predictions','quants']].copy()
    df_quants[ts] = df_quants['actuals'] < df_quants['predictions']
    df_quants_ts.append(df_quants.groupby('quants')[ts].mean())

df_quants_ts = pd.concat(df_quants_ts, axis=1).T
dfs_all = pd.concat(dfs_all)
dfs_all_fixed = pd.concat(dfs_all_fixed)
print("CRPS random quants", crps_rnd.compute().cpu().numpy())
print("CRPS mandatory quants", crps_fixed.compute().cpu().numpy())
print("MAPE", mape.compute().cpu().numpy())
print("MARFE", marfe(dfs_all, bin_size=0.01))


  0%|          | 0/35 [00:00<?, ?it/s]

ValueError: No objects to concatenate

In [ ]:
df_quants_ts.to_excel('results/df_quants_ts.xlsx')
df_quants_ts.mean()

In [ ]:
marfe(dfs_all, bin_size=0.01)

In [ ]:
marfe(dfs_all_fixed, bin_size=0.01)

In [ ]:
np.mean(marfe_ts)

In [ ]:
np.abs(df_quants_ts - np.array(df_quants_ts.columns)[None]).values.mean()

In [ ]:
df_Grzegorz = pd.read_excel('results/Grzegorz/RF.xlsx', header=None)
df_Grzegorz.columns = df_quants_ts.columns[1:-1]

np.abs(df_Grzegorz - np.array(df_Grzegorz.columns)[None]).values.mean()

## Analyze Pawel's results

In [ ]:
root_path = os.path.join(RESULTS_DIR, 'pawel')
algo = 'dotm'
bin_size = 0.001

dfs_all = []
marfe_ts = []
for ts in tqdm(range(35)):
    df = pd.read_pickle(os.path.join(root_path, f'{algo}_{ts}.pickle'))
    df = df[~df.actuals.isnull()]

    mandatory_quants = df.quants.isin(np.array(MANDATORY_TESTING_QUANTS))
    df_rnd = df[~mandatory_quants]

    dfs_all.append(df_rnd)
    
    marfe_ts.append(marfe(df_rnd, predictions_col="aggForec", bin_size=bin_size))

dfs_all = pd.concat(dfs_all)

print(f"MARFE {algo}", marfe(dfs_all, predictions_col="aggForec", bin_size=bin_size))

In [ ]:

MARFE dotm 0.0207857571455102
MARFE snaive 0.03546889814622456
MARFE arima 0.025583501139113167
MARFE mlp 0.01721498534369242

In [ ]:
ls results/pawel

In [ ]:
np.mean(marfe_ts)

In [ ]:
dfs_all.quants.hist(bins=1000)

In [ ]:
dfs_all.quants.hist(bins=1000)